In [1]:
import cv2
import dlib
import time
import pygame

# Initialize pygame mixer
pygame.mixer.init()

# Load face detector and facial landmark predictor
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat') # Path to facial landmark predictor file (.dat format)

# Function to calculate eye aspect ratio
def eye_aspect_ratio(eye_points):
    # Vertical distances
    vertical_1 = ((eye_points[1].x - eye_points[5].x) ** 2 + (eye_points[1].y - eye_points[5].y) ** 2) ** 0.5
    vertical_2 = ((eye_points[2].x - eye_points[4].x) ** 2 + (eye_points[2].y - eye_points[4].y) ** 2) ** 0.5

    # Horizontal distance
    horizontal = ((eye_points[0].x - eye_points[3].x) ** 2 + (eye_points[0].y - eye_points[3].y) ** 2) ** 0.5

    # Eye aspect ratio
    ear = (vertical_1 + vertical_2) / (2.0 * horizontal)
    return ear

# Function to detect closed eyes
def detect_closed_eyes(frame, landmarks):
    left_eye_points = [landmarks.part(i) for i in range(36, 42)]
    right_eye_points = [landmarks.part(i) for i in range(42, 48)]

    left_ear = eye_aspect_ratio(left_eye_points)
    right_ear = eye_aspect_ratio(right_eye_points)

    # Average eye aspect ratio
    ear = (left_ear + right_ear) / 2.0

    # Draw eyes
    for i in range(36, 48):
        cv2.circle(frame, (landmarks.part(i).x, landmarks.part(i).y), 1, (255, 255, 255), -2)

    return ear

# Function to detect yawning
def detect_yawn(frame, landmarks, mouth_open_threshold):
    # Get the mouth region from landmarks
    mouth_points = [landmarks.part(i) for i in range(48, 68)]

    if len(mouth_points) == 20:
        # Calculate the aspect ratio of the mouth
        mouth_width = abs(mouth_points[6].x - mouth_points[0].x)
        mouth_height = abs(mouth_points[3].y - mouth_points[9].y)
        aspect_ratio = mouth_height / mouth_width

        # Draw lines or dots around the mouth
        for i in range(20):
            cv2.circle(frame, (mouth_points[i].x, mouth_points[i].y), 1, (255, 255, 255), -1)
            cv2.line(frame, (mouth_points[i].x, mouth_points[i].y), (mouth_points[(i + 1) % 20].x, mouth_points[(i + 1) % 20].y),
                     (255, 255, 255), ) # Draw a line between the mouth points
        cv2.line(frame, (mouth_points[0].x, mouth_points[0].y), (mouth_points[19].x, mouth_points[19].y), (255, 255, 255, 0), 1)

        return aspect_ratio > mouth_open_threshold

    return False

# Open video capture (replace 'path_to_video' with 0 for webcam detection)
cap = cv2.VideoCapture("Driver_Drowsiness.mp4")

# Set the window name
window_name = "Fatigue Detection"

# Create a resizable window
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

# Set the initial window size
cv2.resizeWindow(window_name, 640, 480)

# Initialize variables for eye detection and score
closed_eye_start = None
closed_eye_alert_threshold = 0.5  # Number of seconds with closed eyes required to show the alert

eye_closed_count = 0  # Counter variable for eye closed detections
yawn_count = 0  # Counter variable for yawn detections
yawning = False  # Flag to indicate if a yawn is in progress

# Load the sound files
sound_file_alert = "elevator-tone-2863.wav" 
sound_alert = pygame.mixer.Sound(sound_file_alert)

# Flag to check if the alert has been given
alert_given = False

# Flag to check if the audio alert is playing
audio_alert_playing = False
audio_alert_start_time = None
audio_alert_duration = 1  # Number of seconds to play the audio alert

# Initialize variables for counting seconds and detections
start_time = time.time()

# Process video frames until the user exits
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the frame in the resized window
    cv2.imshow(window_name, frame)

    # Detect faces using dlib's face detector
    faces = face_detector(gray)

    for face in faces:
        # Detect landmarks in the face region
        landmarks = landmark_predictor(gray, face)

        # Detect closed eyes
        ear = detect_closed_eyes(frame, landmarks)

        # Check if eyes are closed
        ear_threshold = 0.25  # Adjust this value as desired
        if ear < ear_threshold:
            if closed_eye_start is None:
                closed_eye_start = time.time()
            else:
                closed_eye_duration = time.time() - closed_eye_start

                # Check if the closed eye duration exceeds the alert threshold
                if closed_eye_duration >= closed_eye_alert_threshold and not alert_given:
                    cv2.putText(frame, "Drowsiness Detected", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                    sound_alert.play()  # Play the alert sound
                    alert_given = True
                    eye_closed_count += 1  # Increment eye closed count

                # Check if the closed eye duration exceeds the audio alert duration
                if closed_eye_duration >= audio_alert_duration and not audio_alert_playing:
                    sound_alert.play()  # Play the alert sound
                    audio_alert_playing = True
                    audio_alert_start_time = time.time()

                # Check if the audio alert duration has exceeded and repeat the audio alert
                if audio_alert_playing and time.time() - audio_alert_start_time >= audio_alert_duration:
                    sound_alert.play()  # Play the alert sound
                    audio_alert_start_time = time.time()

        else:
            closed_eye_start = None # Reset closed eye start timer
            alert_given = False
            audio_alert_playing = False

        # Detect yawning
        yawn_threshold = 1.5  # Adjust this value as desired
        if detect_yawn(frame, landmarks, yawn_threshold):
            if not yawning:
                cv2.putText(frame, "Yawn Detected", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                yawn_count += 1  # Increment detection count
                yawning = True
        else:
            yawning = False

        # Draw a rectangle around the face
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display the detection counts on the frame
    text1 = f"Eye Closed Count: {eye_closed_count}"
    text2 = f"Yawn Count: {yawn_count}"

    # Determine the text size
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    thickness = 1
    text1_size, _ = cv2.getTextSize(text1, font, font_scale, thickness)
    text2_size, _ = cv2.getTextSize(text2, font, font_scale, thickness)

    # Calculate the box dimensions based on text size
    box1_width = text1_size[0] + 10
    box1_height = text1_size[1] + 10
    box1_coords = (20, frame.shape[0] - box1_height - 40)  # Adjust the y-coordinate to position it at the bottom

    box2_width = text2_size[0] + 10
    box2_height = text2_size[1] + 10
    box2_coords = (20, frame.shape[0] - box2_height - 10)  # Adjust the y-coordinate to position it at the bottom

    # Draw white boxes around the text
    cv2.rectangle(frame, box1_coords, (box1_coords[0] + box1_width, box1_coords[1] + box1_height), (255, 255, 255), cv2.FILLED)
    cv2.rectangle(frame, box2_coords, (box2_coords[0] + box2_width, box2_coords[1] + box2_height), (255, 255, 255), cv2.FILLED)

    # Draw the text on the white boxes
    cv2.putText(frame, text1, (box1_coords[0] + 5, box1_coords[1] + text1_size[1] + 5), font, font_scale, (0, 0, 0), thickness)
    cv2.putText(frame, text2, (box2_coords[0] + 5, box2_coords[1] + text2_size[1] + 5), font, font_scale, (0, 0, 0), thickness)

    # Calculate and display elapsed time
    elapsed_time = time.time() - start_time
    cv2.putText(frame, f"Elapsed Time: {elapsed_time:.2f} seconds", (350, 450), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

    cv2.imshow('Fatigue Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()

pygame 2.4.0 (SDL 2.26.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html
